<a href="https://colab.research.google.com/github/inviduka/SINVI/blob/main/ThInKare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏥 Thinkare MVP — Supabase Colab Notebook
This notebook helps you set up and test the **Thinkare** healthcare MVP backend using **Supabase** + Python SDK.

⚠️ Use this only for backend testing. The React frontend should be developed locally or on Vercel.


In [ ]:
!pip install supabase

In [ ]:
from supabase import create_client

# 🔑 Replace with your Supabase project credentials
url = "https://YOUR_PROJECT_ID.supabase.co"
key = "YOUR_SUPABASE_ANON_KEY"

supabase = create_client(url, key)
print("✅ Supabase client initialized")


In [ ]:

# Try selecting from 'profiles' table (will be empty initially)
try:
    resp = supabase.table("profiles").select("*").execute()
    print("Profiles:", resp)
except Exception as e:
    print("Table not created yet. Run schema in Supabase SQL Editor.")


## 🔐 Auth — Sign up and Sign in test

In [ ]:

email = "village1@test.com"
password = "password123"

# Sign up user
resp = supabase.auth.sign_up({"email": email, "password": password})
print("Sign-up response:", resp)

# Sign in user
resp = supabase.auth.sign_in_with_password({"email": email, "password": password})
print("Sign-in response:", resp)
user = resp.user


## 👤 Create Profile for user

In [ ]:

if user:
    profile_payload = {
        "auth_id": user.id,
        "role": "village",
        "name": "Village One",
        "phone": "9999999999"
    }
    resp = supabase.table("profiles").insert([profile_payload]).execute()
    print("Inserted profile:", resp)
else:
    print("❌ No user available. Run sign-in first.")


## 🎟️ Tickets Flow

In [ ]:

# Insert a ticket
ticket_payload = {
    "created_by": "PROFILE_UUID",   # replace with profile UUID
    "type": "general",
    "description": "Need vaccination camp",
    "preferred_date": "2025-10-05",
    "status": "pending"
}
resp = supabase.table("tickets").insert([ticket_payload]).execute()
print("Ticket created:", resp)


In [ ]:

# NGO accepts a ticket
resp = supabase.table("tickets").update({
    "status": "accepted",
    "ngo_id": "NGO_PROFILE_UUID"   # replace with NGO profile id
}).eq("id", "TICKET_UUID").execute()
print("Ticket accepted:", resp)


## 🔔 Notifications

In [ ]:

# Insert a notification for a user
notif_payload = {
    "user_id": "VILLAGE_PROFILE_UUID",  # replace with village profile id
    "type": "ticket_accepted",
    "payload": {"ticketId": "TICKET_UUID", "ngoId": "NGO_PROFILE_UUID"}
}
resp = supabase.table("notifications").insert([notif_payload]).execute()
print("Notification inserted:", resp)


In [ ]:

# Fetch notifications for user
resp = supabase.table("notifications").select("*").eq("user_id", "VILLAGE_PROFILE_UUID").execute()
print("Notifications:", resp)
